In [1]:
#import necessary libraries
import pandas as pd
import pymorphy2
import re
import nltk

from nltk.tokenize import sent_tokenize, word_tokenize

from yargy.pipelines import morph_pipeline
#from ipymarkup import show_box_markup

from yargy import rule, Parser, or_
from yargy.predicates import eq, type, normalized, gram

In [2]:
#downlaod dataframe from google disk
url='https://drive.google.com/file/d/1boACz8ab9UytCpi0QhTUFRyS2oyH1MXS/view'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
test = pd.read_csv(url)

In [3]:
#take a look at first seven rows of dataframe
test.head(7)

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
6,0,6,client,Да мы работаем с компанией которая нам подлива...


In [4]:
#how many values in dlg_id column
test.dlg_id.value_counts().sort_values()

4     35
3     53
1     55
2     85
0    109
5    143
Name: dlg_id, dtype: int64

In [5]:
man_test = test[test['role']=='manager'] # to keep only managers speach

In [6]:
#get the all texts from DataBase by dlg_id in one list for further actions
texts = []
nums = [0,1,2,3,4,5] #quantity of dialogs in dlg column
for i in nums:
    text = man_test[man_test['dlg_id']==i] 
    text = sent_tokenize('. '.join(text['text']))
    texts.append(text)
print('Total number of dialogs', len(texts))
print(texts[0])

Total number of dialogs 6
['Алло здравствуйте.', 'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается.', 'Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать.', 'Угу а на что вы обращаете внимание при выборе.', 'Что для вас приоритет.', 'Ну у вас срок заканчивается поэтому мы набрали + ко всему.', 'А так нет не только поэтому просто я обратила внимание и вы пользуетесь виджетами мы там с вами сотрудничали.', 'А если вы 19 являетесь то лучше то идти бесплатно.', 'Ага хорошо.', 'Индивидуальным поэтому не все то есть сотрудничают с кем то сейчас да кому то например не с кем сейчас справляться кто то сравнивал.', 'Да мы услышали вас спасибо за рекомендации.', 'Вот понимаю да но дело в том что в основном да за месяц.', 'Необходимо обзванивать.', 'Хорошо я данную информацию тогда зафиксирую передам будем иметь ввиду.', 'И вам спасибо большое за обратную связь.', 'Всего хорошего до

In [7]:
#extract intoduce phrases and names
introduce = {}
names = {}
for count, text in enumerate(texts):
    introduce_o = [sent for sent in text if 'зовут' in sent]
    introduce[count] = introduce_o

prob_thresh = 0.6 # probability score threshold

morph = pymorphy2.MorphAnalyzer()

for k in range(len(introduce)):
    names[k] = []
    for word in nltk.word_tokenize(''.join(introduce[k])):
        for p in morph.parse(word):
            if 'Name' in p.tag and p.score >= prob_thresh:
                names[k].append(word.title())
print(introduce)
print(names)

{0: ['Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается.'], 1: ['Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается.'], 2: ['Меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там.'], 3: ['Добрый меня максим зовут компания китобизнес удобно говорить.'], 4: [], 5: []}
{0: ['Ангелина'], 1: ['Ангелина'], 2: ['Ангелина'], 3: ['Максим'], 4: [], 5: []}


In [8]:
#find company name
KEY_WORD = rule(eq('компания'))

NOUN = gram('NOUN')
ADJF = gram('ADJF')
NAME = gram('Name')

MODIFIER = ADJF.repeatable()
COMP_NAME=rule(
    MODIFIER.optional(),
    NOUN.optional(),
    MODIFIER.optional()
)

COMPANY = rule(KEY_WORD, COMP_NAME) # strict rule to find only phrase with COMPANY name 
parser = Parser(COMPANY)

company_title = []
company_titles = {}

for count, text in enumerate(texts):
    company_titles[count] = []
    for sent in text:
        matches = list(parser.findall(sent))
        #spans = [_.span for _ in matches]
        #show_box_markup(sent, spans)
        for match in matches:
            company = [x.value for x in match.tokens]
            company = ' '.join(company).title()
            #company_title.append(company)
            company_titles[count].append(company)
print(company_titles)

{0: ['Компания Диджитал Бизнес'], 1: ['Компания Диджитал Бизнес'], 2: ['Компания Диджитал Бизнес'], 3: ['Компания Китобизнес', 'Компания'], 4: [], 5: []}


In [9]:
#find greeting phrase

KEY_WORD = morph_pipeline([
    'привет',
    'здравствуйте',
    'добрый день',
    'добрый вечер'
])

NOUN = gram('NOUN')
ADJF = gram('ADJF')
NAME = gram('Name')

MODIFIER = ADJF.repeatable()
GREET_NAME=rule(
    MODIFIER.optional(),
    NOUN.optional(),
    MODIFIER.optional()
)

GREET = rule(KEY_WORD, GREET_NAME)
parser = Parser(GREET)

greetings = {}

for count, text in enumerate(texts):
    greetings[count] = []
    for sent in text:
        matches = list(parser.findall(sent))
        #spans = [_.span for _ in matches]
        #show_box_markup(sent, spans)
        for match in matches:
            greetings_o = [x.value for x in match.tokens]
            greetings_o = ' '.join(greetings_o).title()
            greetings[count].append(greetings_o)
print(greetings)

{0: ['Здравствуйте'], 1: ['Здравствуйте'], 2: ['Здравствуйте'], 3: ['Добрый День'], 4: [], 5: []}


In [10]:
#find goodbye phrase

KEY_WORD = morph_pipeline([
    'до свидания',
    'пока',
    'всего доброго', 
    'хорошего вечера'
])

NOUN = gram('NOUN')
ADJF = gram('ADJF')
NAME = gram('Name')

MODIFIER = ADJF.repeatable()
BYE_NAME=rule(
    MODIFIER.optional(),
    NOUN.optional(),
    MODIFIER.optional(), 
)

BYE = rule(KEY_WORD, BYE_NAME) 
parser = Parser(BYE)

bye_list = {}

for count, text in enumerate(texts):  
    for sent in text:
        bye_list[count] = []
        matches = list(parser.findall(sent))
        #spans = [_.span for _ in matches]
        #show_box_markup(sent, spans)
        for match in matches:
            goodbyes = [x.value for x in match.tokens]
            goodbyes = ' '.join(goodbyes).title()
            bye_list[count].append(goodbyes)
print(bye_list)

{0: ['До Свидания'], 1: ['До Свидания'], 2: [], 3: ['Всего Доброго'], 4: ['До Свидания'], 5: ['До Свидания Хорошего Вечера']}


In [11]:
#put all information from dicts in final dataframe 
final = pd.DataFrame({'Greeting': [''.join(k) for k in greetings.values()], #1st column
                      'Introducing': [''.join(k) for k in introduce.values()], #2nd column
                     'Name': [''.join(k) for k in names.values()], #3rd column
                     'Company Name': [''.join(k) for k in company_titles.values()], #4th column
                     'Goodbye': [''.join(k) for k in bye_list.values()]}) #5th column

In [12]:
# create a function fill_values() to put values in column Insight based on values in other columns
def fill_values(row):
    if row['Greeting'] == '' :
          return 'False'
    if row['Goodbye'] == '' :
          return 'False'
    return 'True'

In [13]:
final['Insight'] = final.apply (lambda row: fill_values(row), axis=1)

In [14]:
#parsing results
final

,Greeting,Introducing,Name,Company Name,Goodbye,Insight
0,Здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,Ангелина,Компания Диджитал Бизнес,До Свидания,True
1,Здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,Ангелина,Компания Диджитал Бизнес,До Свидания,True
2,Здравствуйте,Меня зовут ангелина компания диджитал бизнес з...,Ангелина,Компания Диджитал Бизнес,,False
3,Добрый День,Добрый меня максим зовут компания китобизнес у...,Максим,Компания КитобизнесКомпания,Всего Доброго,True
4,,,,,До Свидания,False
5,,,,,До Свидания Хорошего Вечера,False
